<a href="https://colab.research.google.com/github/LuciaMartinezGavier/compromiso-social-estudiantil/blob/main/parafrasis_es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generación de paráfrasis en español


Elegir insecto
Opciones: babosa, oruga, hormigas, babositas

In [ ]:
bug = 'oruga'

Obtener los textos que se desean parafrasear

In [ ]:
import requests

# url tiene el link a la descarga de un json con todos los párrafos
urls = {
  'babosa':   'https://drive.google.com/uc?export=download&id=1_0DHmdMzrr_0RCbc1-zxMsUNHZ_yTG4X',
  'oruga':    'https://drive.google.com/uc?export=download&id=1UBRHkv7s39ju6fmthJ7lSfQ41Fy6MF8C',
  'hormiga':  'https://drive.google.com/uc?export=download&id=1U63xv_DPTb4FAI9XKiJ9ly6au7FQhNKm',
  'babosita': 'https://drive.google.com/uc?export=download&id=1RzIe2ejobaGxN3m-nrA6p4hyLqdQxjI1'
}

try:
  bug = requests.get(urls[bug]).json()
except:
  print("An error ocurred")


Utilizar modelo para hacer paráfrasis

In [ ]:
!pip install transformers==4.10.2
!pip install sentencepiece==0.1.96

In [ ]:
# importar modelo de paráfrasis
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("mrm8488/bert2bert_shared-spanish-finetuned-paus-x-paraphrasing")
model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/bert2bert_shared-spanish-finetuned-paus-x-paraphrasing")
import torch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print("device ", device)

In [ ]:
from copy import deepcopy
parafrasis = deepcopy(bug)

for topic in bug:
  for paragraph in bug[topic]:
    context = paragraph
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    text = context + " </s>"

    encoding = tokenizer.encode_plus(
        text,
        max_length = 500,
        padding=True,
        return_tensors = "pt"
    )

    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    model.eval()
    beam_outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=500,
        early_stopping=True,
        num_beams=15,
        num_return_sequences=3,
    )

    for beam_output in beam_outputs:
        sent = tokenizer.decode(beam_output, skip_special_tokens = True,
                                clean_up_tokenization_spaces = True)
        parafrasis[topic].append(sent)


In [ ]:
for topic in parafrasis:
  for j in parafrasis[topic]:
    print(j)
  print("-------------------------")

Generar textos usando los párrafos creados. Un párrafo por tema.

In [ ]:
from itertools import product

#final_text = [t1 +"\n" + t2 +"\n" + t3 +"\n"+ t4 +"\n" + t5 for t1, t2, t3, t4, t5 in product(parafrasis["que"], parafrasis["como"], parafrasis["habitat"], parafrasis["daño"], parafrasis["prevencion"])]
final_text = [
    t1 + "\n" + t2 + "\n" + t3 + "\n"+ t4 + "\n" + t5
    for t1, t2, t3, t4, t5 in product(bug["que"], bug["como"], bug["habitat"],
                                      bug["daño"], bug["prevencion"])
]

import random
for i in range(4):
  num = random.randint(0, len(final_text))
  print(final_text[num] + "\n\n")